%load_ext autotime

### Instructions
1. Seek to improve the one-layer model by changing the internal activation function. This will involve changing A1 to something else, and also changing the derivative involving one or both of dAI and dZ1.
2. Add another layer with your choice of activation function (other than the logistic function) and your choice of internal nodes
3. Add a third layer
4. Add a fourth layer
5. Report on your choices and assessment of the resulting models - you can extend the basic model assessment tools supplied
6. Based on your answers to tasks 1-4, write code that - for this dataset - allows a user to build and run a neural net with a high degree of flexibility in terms of architecture and hyperperameter choices.
7. Explain how advanced optimisers (e.g. Adam) would extend your code

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# Import the Dataset

df = pd.read_csv ('flower2D.csv')
print(df.shape)
print(df.info())
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

(400, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  400 non-null    int64  
 1   X1          400 non-null    float64
 2   X2          400 non-null    float64
 3   Y           400 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 12.6 KB
None


,X1,X2,Y
0,0.005257,-0.303586,0
1,0.996098,3.461645,0
2,0.384404,2.392678,0
3,0.951043,3.282709,0
4,0.091932,1.066689,0


In [30]:
X, y = df[["X1","X2"]], np.array(df["Y"])
n = 300
n_test = X.shape[0] - n
X_train, X_test = X[:n].values, X[n:].values       # shape (300, 2) and (n_test, 2)
y_train, y_test = y[:n].reshape(-1,1), y[n:].reshape(-1,1)  # shape (300,1)

#### Loss & functions

In [35]:
def binary_cross_entropy(Y, Y_hat):
    m = Y.shape[0]
    return -(1/m) * np.sum(Y * np.log(Y_hat + 1e-15) + (1 - Y) * np.log(1 - Y_hat + 1e-15))

def binary_cross_entropy_grad(Y, Y_hat):
    m = Y.shape[0]
    return (Y_hat - Y) / m

#### Neural network design
#### Activation Layers

In [36]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        self.out = 1 / (1 + np.exp(-x))
        return self.out

    def backward(self, grad_output):
        return grad_output * self.out * (1 - self.out)

    def parameters_and_gradients(self):
        return []

class Relu:
    def __init__(self):
        self.out = None

    def forward(self, x):
        self.out = np.maximum(0, x)
        return self.out

    def backward(self, grad_output):
        return grad_output * (self.out > 0)

    def parameters_and_gradients(self):
        return []

#### Linear layer

In [37]:
class Linear:
    def __init__(self, input_dim, output_dim):
        self.weights = np.random.randn(input_dim, output_dim) * np.sqrt(2. / input_dim)
        self.biases = np.zeros((1, output_dim))
        self.input = None
        self.grad_weights = None
        self.grad_biases = None

    def forward(self, x):
        self.input = x
        return np.dot(x, self.weights) + self.biases

    def backward(self, grad_output):
        self.grad_weights = np.dot(self.input.T, grad_output)
        self.grad_biases = np.sum(grad_output, axis=0, keepdims=True)
        return np.dot(grad_output, self.weights.T)

    def parameters_and_gradients(self):
        return [(self.weights, self.grad_weights),
                (self.biases, self.grad_biases)]

#### Neural Network

In [38]:
class NeuralNetwork:
    def __init__(self, layers=[]):
        self.layers = layers

    def add_layer(self, layer):
        self.layers.append(layer)

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, grad_output):
        gradient_input = grad_output
        for layer in reversed(self.layers):
            gradient_input = layer.backward(gradient_input)

    def parameters_and_gradients(self):
        params_and_grads = []
        for layer in self.layers:
            params_and_grads.extend(layer.parameters_and_gradients())
        return params_and_grads

### No hidden layers

In [39]:
nn = NeuralNetwork([
    Linear(X_train.shape[1], 1),  # input_dim = n_features
    Sigmoid()
])

learning_rate = 0.1
num_epochs = 2000

for i in range(num_epochs):
    # Forward pass
    y_hat = nn.forward(X_train)

    # Compute loss
    loss = binary_cross_entropy(y_train, y_hat)

    # Backward pass
    grad_loss = binary_cross_entropy_grad(y_train, y_hat)
    nn.backward(grad_loss)

    # Update parameters
    for param, grad in nn.parameters_and_gradients():
        param -= learning_rate * grad

    if i % 200 == 0:
        print(f"Iteration {i}, loss: {loss:.4f}")

for param, _ in nn.parameters_and_gradients():
    print(param)

Iteration 0, loss: 0.9440
Iteration 200, loss: 0.6103
Iteration 400, loss: 0.6040
Iteration 600, loss: 0.6029
Iteration 800, loss: 0.6026
Iteration 1000, loss: 0.6025
Iteration 1200, loss: 0.6025
Iteration 1400, loss: 0.6025
Iteration 1600, loss: 0.6025
Iteration 1800, loss: 0.6025
[[ 0.09949512]
 [-0.24299735]]
[[-0.67412963]]


In [41]:
# Print the learned weights and bias
for layer in nn.layers:
    if isinstance(layer, Linear):
        print("Weights:\n", layer.weights)
        print("Bias:\n", layer.biases)

Weights:
 [[ 0.09949512]
 [-0.24299735]]
Bias:
 [[-0.67412963]]


In [42]:

y_hat_test = nn.forward(X_test)


predictions = (y_hat_test > 0.5).astype(int)
labels = (y_test > 0.5).astype(int)


predictions = predictions.flatten()
labels = labels.flatten()


tp = np.sum((predictions == 1) & (labels == 1))
tn = np.sum((predictions == 0) & (labels == 0))
fp = np.sum((predictions == 1) & (labels == 0))
fn = np.sum((predictions == 0) & (labels == 1))


cm = np.array([[tn, fp],
               [fn, tp]])

print("Confusion matrix:\n", cm)


model_score = (tn + tp) / (tn + fp + fn + tp)
print("Model score:", model_score)

Confusion matrix:
 [[ 0  0]
 [72 28]]
Model score: 0.28
